<a href="https://colab.research.google.com/github/KonstantinBurkin/Machine_Learning_Project/blob/main/Machine_Learning_Delivery_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sales Forecasting

In this project I build a Machine Learning method that predicts the number of orders for each store in Delivery Club and each product for the next week. Here, I use data set describing the number of sales of goods in stores on the Delivery Club platform in 10 cities in Russia (Moscow, St. Petersburg, Krasnodar, Samara, Nizhny Novgorod, Rostov-on-Don, Volgograd, Voronezh, Kazan, Yekaterinburg).

Two data sets (train.csv and test.csv) are used to train Regression model and evaluate its accuracy. Both data sets have the same list of variables:
- id - уникальный идентификатор, представляющий связку (product_id,
store_id, date). Это значит, что для каждой тройки (product_id, store_id, date) существует лишь один id, он не повторяется в данных
- date - дата продажи продукта
- city_name - название города, в котором происходила продажа
- store_id - уникальный идентификатор для каждого магазина
- category_id - категория продаваемого товара
- product_id - уникальный идентификатор товара
- price - цена товара
- weather_desc - краткое описание погоды в этом городе в день продажи
- humidity - влажность в этом городе в день продажи
- temperature - температура в этом городе в день продажи
- pressure - атмосферное давление в этом городе в день продажи
- sales - количество продаж товара (это то, что нужно прогнозировать)

- MAE - средняя абсолютная ошибка, показывает на сколько заказов в среднем ошибается прогноз. Это метрика очень легко интерпретируема. Если у нас значение метрики равно, например, 5, то это значит модель в среднем ошибается на 5 заказов каждый час по каждому району. Метрика не бывает отрицательной, так как все ошибки беруться по модулю, для идеальной модели эта метрика будет равна 0. Метрика не так чувствительна к выбросам.

$$MAE = \frac1N \sum ^{N}_{i=1} |y_i-\hat y_i|$$

In [108]:
# Import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [127]:
# Upload data sets into the project

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## Data sets description

In [57]:
train = train.replace(
    ('Москва', 'Санкт-Петербург', 'Краснодар', 'Самара','Нижний Новгород', 'Ростов-на-Дону', 'Волгоград', 'Воронеж'),
       ("Moscow", "St.Petersburg", "Krasnodar", "Samara", "Nizhny.Novgorod", "Rostov-on-Don", "Volgograd", "Voronezh")
       )

In [58]:
pd.unique(train["city_name"])
# pd.unique(train["store_id"])
# можно сделать график количества магазинов в каждом городе

array(['Moscow', 'St.Petersburg', 'Krasnodar', 'Samara',
       'Nizhny.Novgorod', 'Rostov-on-Don', 'Volgograd', 'Voronezh', nan],
      dtype=object)

In [7]:
train.shape

(249664, 12)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249664 entries, 0 to 249663
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            249664 non-null  int64  
 1   date          249663 non-null  object 
 2   city_name     249663 non-null  object 
 3   store_id      249663 non-null  float64
 4   category_id   249663 non-null  float64
 5   product_id    249663 non-null  float64
 6   price         249663 non-null  float64
 7   weather_desc  249663 non-null  object 
 8   humidity      249663 non-null  float64
 9   temperature   249663 non-null  float64
 10  pressure      249663 non-null  float64
 11  sales         249663 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 22.9+ MB


In [48]:
train.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure,sales
0,1,2021-07-29,Москва,1.0,1.0,1.0,4.79,"переменная облачность, небольшой дождь",61.9375,23.1875,741.0000,26.0
1,2,2021-07-30,Москва,1.0,1.0,1.0,4.79,"переменная облачность, небольшой дождь",70.2500,22.1875,740.3125,37.0
2,3,2021-07-31,Москва,1.0,1.0,1.0,4.79,переменная облачность,52.6250,21.8125,741.6250,25.0
3,4,2021-08-01,Москва,1.0,1.0,1.0,4.79,"облачно, небольшой дождь",87.4375,20.0625,743.3125,26.0
4,5,2021-08-02,Москва,1.0,1.0,1.0,4.79,переменная облачность,66.1875,23.4375,739.6250,22.0


In [49]:
train.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure,sales
count,249664.000000,249663.000000,249663.000000,249663.000000,249663.000000,249663.000000,249663.000000,249663.000000,249663.000000
mean,124831.500008,28.702791,2.433785,17.882938,5.080392,73.242757,5.719003,751.724507,10.294261
std,72071.933127,16.405095,1.927403,10.790011,3.405103,20.673135,11.564482,9.361237,14.526384
min,1.000000,1.000000,1.000000,1.000000,1.930000,13.750000,-24.000000,717.250000,0.000000
25%,62415.750000,15.000000,1.000000,9.000000,3.000000,58.250000,-2.312500,745.687500,2.000000
50%,124831.500000,29.000000,1.000000,17.000000,4.090000,78.125000,5.375000,751.312500,6.000000
75%,187247.250000,43.000000,4.000000,28.000000,6.020000,91.937500,12.750000,758.000000,12.000000
max,249663.000000,57.000000,9.000000,35.000000,18.630000,100.000000,34.250000,779.000000,269.000000


In [17]:
test.shape

(24836, 11)

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24836 entries, 0 to 24835
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            24836 non-null  int64  
 1   date          24836 non-null  object 
 2   city_name     24836 non-null  object 
 3   store_id      24836 non-null  int64  
 4   category_id   24836 non-null  int64  
 5   product_id    24836 non-null  int64  
 6   price         24836 non-null  float64
 7   weather_desc  24836 non-null  object 
 8   humidity      24836 non-null  float64
 9   temperature   24836 non-null  float64
 10  pressure      24836 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 2.1+ MB


In [9]:
test.head()

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure
0,666677,2022-02-14,Москва,1,1,1,4.79,облачно,87.3125,-1.9375,749.3125
1,666678,2022-02-15,Москва,1,1,1,4.79,переменная облачность,88.7500,-1.2500,752.6875
2,666679,2022-02-16,Москва,1,1,1,4.79,переменная облачность,90.3750,-1.5625,746.3125
3,666680,2022-02-17,Москва,1,1,1,4.79,"облачно, небольшой дождь",98.0000,1.7500,732.6875
4,666681,2022-02-18,Москва,1,1,1,4.79,"облачно, небольшие осадки",95.5000,1.3750,733.0000


In [12]:
test.describe()

,id,store_id,category_id,product_id,price,humidity,temperature,pressure
count,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000,24836.000000
mean,679094.500000,78.053551,2.375423,17.832300,5.201144,87.285168,-0.751719,747.908286
std,7169.679979,45.689019,1.876578,10.826993,3.491933,9.839292,4.059063,9.743387
min,666677.000000,1.000000,1.000000,1.000000,1.930000,55.875000,-10.500000,730.312500
25%,672885.750000,40.000000,1.000000,9.000000,3.000000,84.812500,-3.625000,740.000000
50%,679094.500000,76.000000,1.000000,17.000000,4.090000,89.937500,-0.312500,748.937500
75%,685303.250000,117.000000,4.000000,28.000000,6.020000,94.312500,1.750000,754.687500
max,691512.000000,164.000000,9.000000,35.000000,18.630000,98.625000,9.062500,769.000000


In [53]:
group.head()

,date,city_name,sales
0,2021-07-29,Волгоград,693.0
1,2021-07-29,Воронеж,1086.0
2,2021-07-29,Краснодар,1271.0
3,2021-07-29,Москва,3755.0
4,2021-07-29,Нижний Новгород,818.0


In [59]:
# график заказов по городам
group = train[['date', 'city_name', 'sales']].groupby(['date', 'city_name'], as_index=False).sum()
fig = px.line(group, x="date", y="sales", color='city_name', template='plotly_dark')
fig.update_layout(
    title={
        'text': "Orders in each city",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show() 
# можно еще добавить инфу как менялись заказы по месяцам, в течении недели

## Building linear regression model

In [128]:
# make 4 subsets for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    train.iloc[:,0:11],
    train.iloc[:,11],
    train_size = 0.8, 
    test_size = 0.2,
    random_state = 2022,
    shuffle = True)


In [152]:
# Linear model

model = LinearRegression()
model.fit(X_train.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False), # выкидываю character data type and NA
          y_train.dropna(axis=0, how='any', inplace=False)) # выкидываю character data type and NA

forecast = model.predict(X_test.iloc[:,[0,3,4,5,6,8,9,10]].dropna(axis=0, how='any', subset=None, inplace=False))
mae = mean_absolute_error(y_test, forecast)

print(f"MAE = {mae:.2f} < 4.10 " if mae < 4.10 else f"MAE = {mae:.2f} > 4.10 ")


MAE = 8.56 > 4.10 


## Results

In [ ]:
# df.to_csv("prediction.csv")